In [55]:
#%load_ext autoreload
#%autoreload 2
import matplotlib as mpl
import scipy as sp
import numpy as np
import pandas as pd
# sometimes order maters on these
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import oBehave.plotting_stuff as obps
import oBehave.helperfunctions as obhf
import oBehave.neural_analysis as obna
import oBehave.behavior_anaylsis as obba
import sklearn.decomposition as decomp
from sklearn.model_selection import train_test_split
import sklearn.decomposition as decomp
from sklearn.preprocessing import StandardScaler

from visual_behavior.ophys.dataset.visual_behavior_ophys_dataset import VisualBehaviorOphysDataset
from visual_behavior.ophys.response_analysis.utilities import get_trace_around_timepoint,get_nearest_frame
from visual_behavior.ophys.response_analysis.response_analysis import ResponseAnalysis

#from dPCA import dPCA
# Load the manifest
manifest = obhf.load_manifest()
obba.includeNovelSession(manifest);

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [40]:
experiment_id = 639438856# experiment B
dataset= VisualBehaviorOphysDataset(experiment_id=experiment_id, cache_dir=obhf.drive_path)
analysis = ResponseAnalysis(dataset)

loading trial response dataframe
loading flash response dataframe


In [36]:
def killnancells(dataset,analysis):
    bad_cell_indices = []
    bad_cell_ids = []
    for cell_index in range(dataset.dff_traces[1].shape[0]):
        if np.isnan(dataset.dff_traces[1][cell_index][0]): #check whether first datapoint is nan
           bad_cell_indices.append(cell_index)
           bad_cell_ids.append(dataset.get_cell_specimen_id_for_cell_index(cell_index))
    fdf = analysis.flash_response_df.copy()
    fdf = fdf[fdf.cell.isin(bad_cell_indices)==False]
    return fdf

def getResponseArray(dataset,analysis):
    # We want to bootstrap dim analysis:
    num_trials = len(dataset.stimulus_table)
    fr = killnancells(dataset,analysis)
    use_cells = np.unique(fr.cell)
    response_array = np.empty((num_trials,len(use_cells)))
    for i in range(num_trials):
        response_array[i,:] = fr[fr.flash_number==i].mean_response.values
    return response_array,fr # return response array and flash response data frame

In [46]:
fr = obba.include_add_repeats(analysis.flash_response_df,dataset.all_trials)

In [54]:
fr.response_type

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
5          NaN
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12         NaN
13         NaN
14         NaN
15         NaN
16         NaN
17         NaN
18         NaN
19         NaN
20         NaN
21         NaN
22         NaN
23         NaN
24         NaN
25        MISS
26         NaN
27         NaN
28         NaN
29         NaN
          ... 
378064     NaN
378065     NaN
378066     NaN
378067     NaN
378068     NaN
378069     NaN
378070     NaN
378071     NaN
378072     NaN
378073     NaN
378074    MISS
378075     NaN
378076     NaN
378077     NaN
378078     NaN
378079     NaN
378080     NaN
378081     NaN
378082     NaN
378083     NaN
378084     NaN
378085    MISS
378086     NaN
378087     NaN
378088     NaN
378089     NaN
378090     NaN
378091     NaN
378092     NaN
378093     NaN
Name: response_type, Length: 378094, dtype: object

In [44]:
fr = obba.include_add_repeats(dataset.stimulus_table,dataset.all_trials)
# Compute the response array
num_trials = len(dataset.stimulus_table)
fr = killnancells(dataset,analysis)
X = fr.merge(fb,left_on = ('flash_number','start_time','image_name'),
             right_on = ('flash_number','start_time','image_name'),how= 'left') 
X_tmp = X[X.repeats==1]
use_cells = np.unique(X_tmp.cell)
first_trials = np.unique(X_tmp.flash_number[X_tmp.repeats==1])
response_array_first = np.empty((len(first_trials),len(use_cells)))
first_flashes = X_tmp[X_tmp.repeats==1].flash_number.values
for i,f in enumerate(first_trials):
    response_array_first[i,:] = X_tmp[X_tmp.flash_number==f].mean_response.values

KeyError: ('cell', u'occurred at index 0')